In [64]:
# make imports
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

In [65]:
# read datasets (disasters)
carbon_df = pd.read_csv('Data/carbon-emissions.csv')
disasters_df = pd.read_csv('Data/natural-disaster-data/number-of-natural-disaster-events.csv')
econ_df = pd.read_csv('Data/natural-disaster-data/economic-damage-from-natural-disasters.csv')

# Q1: Given a year is between 2000-2018, what is the chance that it's number of total disasters is greater than the average number of disasters for all years 1900-2018?

# Bayes' Theorem
## $ P(A|B) = \frac{P(A and B)}{P(A)} $

### In our scenario:

### $ P(A|B) $ = P(Year Has More Natural Disasters than the Mean for All Years 1900-2018, Given Year is 2000-2018)

### $ P(A) $ = P(Year is Between 2000-2018) 

## Getting a "Feel" For the Data

In [66]:
disasters_df.head()

,Entity,Code,Year,Number of reported natural disasters (reported disasters)
0,All natural disasters,NaN,1900,5
1,All natural disasters,NaN,1901,2
2,All natural disasters,NaN,1902,9
3,All natural disasters,NaN,1903,8
4,All natural disasters,NaN,1904,2


# What is the mean number of total natural diasters annually, for all years 1900-2018?

In [89]:
# find all rows reporting "all natural disasters"
COUNT = 'Number of reported natural disasters (reported disasters)'
all_disasters = disasters_df[disasters_df['Entity'] == 'All natural disasters'][COUNT]
# sum them together, divide by their number
mean_disasters = np.sum(all_disasters) / len(all_disasters)
# print the mean
mean_disasters

111.02521008403362

# How Many Years Between 1900-2018 Have More Than This Mean?

In [68]:
count = 0
for num in all_disasters:
    if num > mean_disasters:
        count += 1
count

42

# Do all years 2000-2018 have more total disasters than the mean?

In [69]:
all_disasters_years_and_counts = disasters_df[(disasters_df['Entity'] == 'All natural disasters')]
years_2000_2018 = all_disasters_years_and_counts.tail(19)
count = 0
for num in years_2000_2018['Number of reported natural disasters (reported disasters)']:
    if num > mean_disasters:
        count += 1
        
print(round((count/19) * 100, 2))  # have all these years surpassed the mean we calculated?

100.0


# So, What's the Chance that a Year has an Above Average Number of Natural Disasters, given the year is 2000-2018?

In [70]:
PERCENT = round((18/42) * 100, 2)
print(f'{round((18/42) * 100, 2)}%')

42.86%


# Q2: What is the probability distribution of natural disasters over the years 1900-1999? 2000-2018?

In [71]:
# let's take another look at that data
all_disasters_years_and_counts

,Entity,Code,Year,Number of reported natural disasters (reported disasters)
0,All natural disasters,NaN,1900,5
1,All natural disasters,NaN,1901,2
2,All natural disasters,NaN,1902,9
3,All natural disasters,NaN,1903,8
4,All natural disasters,NaN,1904,2
...,...,...,...,...
114,All natural disasters,NaN,2014,320
115,All natural disasters,NaN,2015,380
116,All natural disasters,NaN,2016,325
117,All natural disasters,NaN,2017,276


In [79]:
years = all_disasters_years_and_counts['Year']
counts = all_disasters_years_and_counts['Number of reported natural disasters (reported disasters)']

# Q3: What % of natural disasters happen before 1925? 1950? 1975? 2000? 2018? (CDF)

# Q4: By the decade: How does the mean number of annual natural disasters change?

# Q5: By the decade:  How does the median number of annual natural disasters change?¶

# Q6: At what number of natural disasters, did 75% of years 1900-2000 have less than that? Of years 2000-2018?

# Q7: At what economic cost, did 75% of years 1900-2000 experience than that (due to natural disasters)? Of years 2000-2018?

# Q8 Null Hypothesis: A greater average number of CO2 emissions from 2000-2018 is responsible from greater mean number of natural disasters annually from 2000-2018.

# Do we accept or reject this?

# Summary of Findings

# 1. Years between 2000-2018 make up nearly 43% of all years between 1900-2018 that have an above average* number of total natural disasters.

## Average taken for years 1900-2018.